In [10]:
# http_client_tests.py
# JupyterLab-compatible test harness for fetch_result()

from http_client import fetch_result  # 模块化后的导入
import sys
import logging

# 各测试用例格式: (描述, URL, 是否期望成功)
test_cases = [
    ("✅ Basic 200 OK", "http://insecure.stevetarzia.com/basic.html", True),
    ("🔁 301 Redirect to HTTPS (not supported)", "http://airbedandbreakfast.com", False),
    ("🔁 302 Redirect chain", "http://maps.google.com", False),
    ("📦 URL with port number", "http://portquiz.net:8080/", True),
  #这个网址出问题了  ("❌ 404 Not Found", "http://cs.northwestern.edu/340", False), 
  #这个网址出问题了  ("♾️ Infinite redirect hell", "http://insecure.stevetarzia.com/redirect-hell", False),
  #  ("🖼 Non-HTML content", "http://insecure.stevetarzia.com/image.png", False),
  #  ("📄 Large HTML page", "http://insecure.stevetarzia.com/libc.html", True),
   ("⚠️ Missing trailing slash", "http://insecure.stevetarzia.com", True),  #这个网址出问题了，所以测试失败
    ("⚠️ Missing trailing slash", "http://www.google.com", True),
    ("🌍 No Content-Length", "http://google.com", True)
]

def run_test(description, url, expect_success):
    print(f"\n=== {description} ===")
    print(f"URL        : {url}")

    # ✅ 强制刷新 logging 设置，让输出流绑定在 stdout（确保 print 顺序一致）
    logging.basicConfig(
        level=logging.DEBUG,
        format='[%(levelname)s] %(message)s',
        stream=sys.stdout,   # ← 输出去 stdout，而不是默认 stderr
        force=True           # ← Python 3.8+：强制覆盖旧设置
    )
    try:
        # 调用主函数获取结构化结果
        result = fetch_result(url, debug=True)

        print("STATUS    :", result.get("status_code"))
        print("EXIT CODE :", result.get("exit_code"))
        print("ERROR MSG :", result.get("error_msg", "<None>"))
        print("CONTENT ↓↓↓")
        print(result.get("content", "")[:300] + ("..." if len(result.get("content", "")) > 300 else ""))

        actual_success = result.get("exit_code", 1) == 0
        if actual_success == expect_success:
            print("✅ TEST PASSED")
        else:
            print("❌ TEST FAILED")
    except Exception as e:
        print(f"❌ Exception: {e}")
        print("❌ TEST FAILED")

# 执行所有测试案例
for desc, url, expect in test_cases:
    run_test(desc, url, expect)



=== ✅ Basic 200 OK ===
URL        : http://insecure.stevetarzia.com/basic.html
[DEBUG] Connecting to insecure.stevetarzia.com:80
[DEBUG] Sending request:
GET /basic.html HTTP/1.0
Host: insecure.stevetarzia.com


STATUS    : 200
EXIT CODE : 0
ERROR MSG : 
CONTENT ↓↓↓
<!DOCTYPE html>
<html>
<body>

<h1>My First Heading</h1>
<p>My first paragraph.</p>

</body>
</html>

✅ TEST PASSED

=== 🔁 301 Redirect to HTTPS (not supported) ===
URL        : http://airbedandbreakfast.com
[DEBUG] Connecting to airbedandbreakfast.com:80
[DEBUG] Sending request:
GET / HTTP/1.0
Host: airbedandbreakfast.com


STATUS    : 301
EXIT CODE : 1
ERROR MSG : Redirected to HTTPS not supported: https://www.airbnb.com/belong-anywhere
CONTENT ↓↓↓

✅ TEST PASSED

=== 🔁 302 Redirect chain ===
URL        : http://maps.google.com
[DEBUG] Connecting to maps.google.com:80
[DEBUG] Sending request:
GET / HTTP/1.0
Host: maps.google.com


[WARNING] Redirected to: http://maps.google.com/maps
[DEBUG] Connecting to maps.google.com: